Wikipedia library example 

In [168]:
import requests
import typing

session = requests.Session()

In [169]:
# text,         type:str    - Content wanting to be summarized.

# Note: Error occurs when text is too short.
def summary(text:str)-> dict:
  API_KEY = "*******"
  url = "https://api.smmry.com"
  
  data = {"sm_api_input":text}
  params = {"SM_API_KEY":API_KEY}
  header_params = {"Expect":"100-continue"}

  res = session.post(url=url, params=params, data=data, headers=header_params).json()
  try:
    return {'status':200, 'message':res['sm_api_content']}
  except Exception:
    return {'status':500, 'message':res['sm_api_message']}

In [173]:
# name,         type:str  - First and last name of a given actor
# num_sentence, type:int  - Number of sentences returned for wikipedia api

# Note: num_sentence should range between 4-10
def search_wiki(name:str, num_sentence:int=10) -> dict:
  url = "https://en.wikipedia.org/w/api.php?&exintro&explaintext&indexpageids"
  params = {
    "action":"query",
    "format":"json",
    "titles":name,
    "exsentences":num_sentence,
    "prop":"extracts|info",
    "redirects":1,
    "inprop":"url"
  }
  res = session.get(url=url, params=params).json()
  try:
    # Return dictionary of responses
    pageid = res['query']['pageids'][0]
    content = str(res['query']['pages'][str(pageid)]['extract'])
    sum = summary(text=content)
    return {'status':200, 
            'response':{
                'source':res['query']['pages'][str(pageid)]['fullurl'],
                'title':res['query']['pages'][str(pageid)]['title'],
                'summary':sum['message'] if sum['status'] is 200 else content
                }
            }
  except KeyError:
    # Person not found
    return {'error':404, 'message':f'Could not find \"{name}\".'}
  except Exception as e:
    # Generic Exception
    return {'error':500, 'message':f"Error processing request.{e}"}



Note, the above function does not work when name is misspelled. For our implementation, we will not worry about misspell since data will be read from classifier.

In [174]:
name = "bob marley"
response = search_wiki(name)
print(response)

{'response': {'source': 'https://en.wikipedia.org/wiki/Bob_Marley',
  'summary': 'Robert Nesta "Bob" Marley, was a Jamaican singer, songwriter, and musician. Marley\'s contributions to music increased the visibility of Jamaican music worldwide, and made him a global figure in popular culture for over a decade. Over the course of his career Marley became known as a Rastafari icon, and he infused his music with a sense of spirituality. Born in Nine Mile, British Jamaica, Marley began his professional musical career in 1963, after forming Bob Marley and the Wailers. The group released its debut studio album The Wailing Wailers in 1965, which contained the single "One Love/People Get Ready"; the song was popular worldwide, and established the group as a rising figure in reggae. The Wailers subsequently released eleven further studio albums; while initially employing louder instrumentation and singing, the group began engaging in rhythmic-based song construction in the late 1960s and early 